# crazyswarm-demo
Prototype code for the crazyswarm demo for the AERO department.

In [1]:
from functools import reduce

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
from scipy.optimize import linear_sum_assignment

In [2]:
%matplotlib ipympl

In [3]:
f3 = plt.figure()
ax3 = f3.add_subplot(projection='3d')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
n_crazyflies = 9
n_dim = 3

### define base waypoints

In [5]:
theta = np.arange(0, 2*np.pi, 2*np.pi/n_crazyflies)
theta_c = np.linspace(np.pi/2, 3*np.pi/2, n_crazyflies)
W = 3
H = 4

formations = {
    'start': np.vstack([
        np.zeros(n_crazyflies),
        np.linspace(-W, W, n_crazyflies),
        np.zeros(n_crazyflies)
    ]),
    'I': np.vstack([
        [-W,  W, -W, W, *[0] * (n_crazyflies - 4)],
        np.zeros(n_crazyflies),
        [-H, -H,  H, H, *np.linspace(-H, H, n_crazyflies-4).tolist()]
    ]),
    'C': np.vstack([
        W * np.cos(theta_c).round(3) + 1.67,
        np.zeros(n_crazyflies),
        H * np.sin(theta_c).round(3)
    ]),
    'O': np.vstack([
        W * np.cos(theta).round(3),
        np.zeros(n_crazyflies),
        H * np.sin(theta).round(3)
    ]),
    'N': np.vstack([
        [-W, -W, W, W, *np.linspace(-W, W, n_crazyflies-4).tolist()],
        np.zeros(n_crazyflies),
        [-H, 0, 0, H, *np.linspace(H, -H, n_crazyflies-4).tolist()]
    ])
}

# Raise the Z's to some height above the ground.
Z_BASE = 1
lowest_z = reduce(lambda m, form: form[2].min(), formations.values())
for formation in formations.values():
    formation[2] += Z_BASE - lowest_z

In [13]:
def plot_formo(k):
    """Render the given formation on the axes"""
    ax3.scatter(formations[k][0], formations[k][1], formations[k][2])

ax3.cla()

plot_formo('start')
# plot_formo('I')
# plot_formo('C')
# plot_formo('O')
# plot_formo('N')

### assign drones to each waypoint

In [14]:
order = ['start', 'I', 'C', 'O', 'N']
base_waypoints = np.zeros((len(order), n_dim, n_crazyflies))
base_waypoints[0] = formations[order[0]]

# Compute base waypoints via linear sum assignment to minimize overlaps.
for i in range(len(order)-1):
    # Find the best association to the next formation to minimize the sum of squared distances.
    distances = np.linalg.norm(
        np.moveaxis(base_waypoints[i, ..., np.newaxis], 0, 2) 
      - np.swapaxes(formations[order[i+1]][..., np.newaxis], 0, 2), 
    axis=2)
    start_ind, end_ind = linear_sum_assignment(distances, maximize=False)

    base_waypoints[i+1, :, start_ind] = formations[order[i+1]][:, end_ind].T

In [15]:
ax3.cla()

cs = plt.cm.tab20.colors[::2]
for i_agent in range(n_crazyflies):
    ax3.scatter(
        base_waypoints[:, 0, i_agent], 
        base_waypoints[:, 1, i_agent], 
        base_waypoints[:, 2, i_agent], 
        color=cs[i_agent])

### add waypoints between waypoints to allow for more defined transitions

In [16]:
trans_len = 10
hold_len = 5
wp_len = trans_len + hold_len
waypoints = np.zeros((len(order) * wp_len, n_dim, n_crazyflies))
waypoints[-wp_len:] = base_waypoints[-1]

for i_wp in range(len(order)-1):
    for i_dim in range(n_dim):
        for i_agent in range(n_crazyflies):
            # print(slice(i_wp*wp_len, i_wp*wp_len + trans_len),
                  # slice(i_wp*wp_len + trans_len, (i_wp+1)*wp_len))
            waypoints[i_wp*wp_len : i_wp*wp_len + trans_len, i_dim, i_agent] = np.linspace(
                base_waypoints[i_wp, i_dim, i_agent], 
                base_waypoints[i_wp+1, i_dim, i_agent],
                trans_len)
            waypoints[i_wp*wp_len + trans_len : (i_wp+1)*wp_len, i_dim, i_agent] \
                = base_waypoints[i_wp+1, i_dim, i_agent]

waypoints = waypoints.swapaxes(1, 2)

### write to gif

In [20]:
def animate(i):
    """Write all waypoints into an animation"""
    
    ax3.cla()
    plt.title("ICON Demo")
    cs = plt.cm.tab20.colors[::2]
    for i_agent in range(n_crazyflies):
        ax3.scatter(
            waypoints[i, i_agent, 0], 
            waypoints[i, i_agent, 1], 
            waypoints[i, i_agent, 2], 
            color=cs[i_agent])

    ax3.grid(True)
    ax3.axis([-H+1, H+1, -H+1, H+1])
    ax3.set_zlim(0, 2*H + Z_BASE)
    ax3.view_init(elev=20.0, azim=i*360/N)

In [21]:
N = waypoints.shape[0]
anim = FuncAnimation(f3, animate, frames=N)
anim.save('animation.gif', fps=N//10)

MovieWriter ffmpeg unavailable; using Pillow instead.
